# Herramientas de visualización - Algunos consejos útiles (Parte 2)

En este cuaderno hacemos una exploración por los diversos tipos de gráficos que tenemos a nuestra disposición. Evaluaremos diferentes situaciones y daremos algunos consejos de el tipo de gráfico que mejor se ajuste a los datos analizados. No vamos a establecer unas reglas, en muchas de estas situaciones se apremia la creatividad de la persona que conoce la información, sin embargo, estos tips suelen destrabar ese espiritu creativo.

In [0]:
import numpy as np
import pandas as pd
import dataprep.eda as dp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, Layout
import ipywidgets as widgets
from plotly.subplots import make_subplots


## (Caso 5) Contratos en Colombia

Este caso ofrece algunas sugerencias para analizar un conjunto de datos compuesto por una variable numérica medida en grupos y subgrupos.

El conjunto de datos corresponde a una muestra de una base de datos que contiene información acerca de contratos adjudicados a nivel nacional.

Los datos se han recopilado en el sitio web [datosabiertos.gov.co](https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos-ACTIVOS/p8vk-huva) y la muestra de este ejemplo se puede encontrar en [github.com/madcentral/metodosestadisticos](https://github.com/madcentral/metodosestadisticos). 

In [0]:
pd.options.display.float_format = '{:}'.format
df = pd.read_csv('Muestra Contratos.csv',index_col=0)

In [0]:
df

In [0]:
df.dtypes

In [0]:
df.describe()

In [0]:
df.describe(include="object")

Es una base con muchas variables, el siguiente código dividira los diferentes tipos de variables y hara algunas conversiones necesarias. Luego describiremos algunos pasos a realizar, con el fin de responder a la pregunta planteada:

In [0]:
#Separamos cuali de cuanti
cualicolumns=df.select_dtypes(exclude=['int64','float64']).columns
cuanticolumns=df.select_dtypes(include=['int64','float64']).columns
Fechas=[i for i in cualicolumns if i.count('Fecha')>0]
cualicolumns=[i for i in cualicolumns if i.count('Fecha')<=0]

In [0]:
# Cambiamos el tipo de dato para las fechas
for i in Fechas:
    df[i]=pd.to_datetime(df[i])
df[Fechas].dtypes

**¿Cómo es el gasto público por regiones dependiendo del sector?**

Para resolver este ejercicio podemos considerar el Valor de los contratos por sector en cada región del país.

## Treemap interactivo

Plotly nos permite revisar esta información de manera dinámica, usando un treemap:


In [0]:
fig = px.treemap(df, 
                 path= ['Sector','Departamento'], 
                 values='Valor del Contrato',
                 
                )

fig.data[0].textinfo= "label+percent parent+percent entry+percent root"
fig.data[0].hovertemplate = '<b>%{label} </b> <br> Suma de Contratos: %{value:$,.0f}<br> '
fig.show()

### Boxplot
Sin embargo, ante la cantidad considerable de departamentos puede ser una opción poco  saludable, un boxplot agrupado con una herramienta interactiva puede ser una buena alternativa:

In [0]:
def sectorvsdepto(i):
    fbase=df[df['Departamento']==i]
    fig, axs = plt.subplots(figsize=(10,5))
    axs=sns.boxplot(y=fbase['Valor del Contrato'].values,x=fbase['Sector'])
    axs.set_xticklabels(axs.get_xticklabels(), rotation=90);
    axs.grid()
    axs.set_ylim(0,210000000)
    plt.show()
    return 
interact(sectorvsdepto,i=widgets.Dropdown(options=sorted(df['Departamento'].unique()),description="Dpto"))

Para cada sector podemos ver la evolución de contratos por departamento:

In [0]:
df['Año']=pd.DatetimeIndex(df['Fecha de Inicio del Contrato']).year

In [0]:
def sectorvsdeptovsanio(i,j):
    print(j)
    try:
        df2=df[df['Departamento']==i]
        fbase=df2[df2['Sector'].isin(j)]
        fig=px.box(fbase,x="Año", y="Valor del Contrato", color='Sector',points='all');
        fig.show()
    except:
        print("En este Departamento no invirtieron en ese sector")
        
    return 
interact(sectorvsdeptovsanio,
         i=widgets.ToggleButtons(options=sorted(df['Departamento'].unique()),
                            description="Dpto:",button_style='info'
                        ),
         j=widgets.SelectMultiple(
             options=df['Sector'].unique(),
             description='Sector:'))


In [0]:
DF2=df.groupby(['Departamento','Sector','Año']).sum()
DF2.reset_index(inplace=True)
DF2

In [0]:
def sectorvsdeptovsanio(i):
    fbase=DF2[DF2['Departamento']==i]
    fig = px.line(fbase, x="Año", y="Valor del Contrato", color='Sector')
    fig.show()
    return 
interact(sectorvsdeptovsanio,i=widgets.Dropdown(options=df['Departamento'].unique(),description="Dpto"))

### Gráficas Combinadas

Podriamos contrastar los valores del contrato con  la cantidad de contratos, como sigue:

In [0]:
import matplotlib.gridspec as gridspec
def sectorvsdeptovsanio(i,j):
    try:
        df2=df[df['Departamento']==i]
        fbase=df2[df2['Sector'].isin(j)]        
        fig = plt.figure(tight_layout=True,figsize=(12,5)) ;       
        gs = gridspec.GridSpec(2, 2)
        ax = fig.add_subplot(gs[:,0])
        sns.boxplot(ax=ax,data=fbase,x='Año',y='Valor del Contrato',hue='Sector')
        plt.grid()
        ax1=fig.add_subplot(gs[0,1])
        base2=fbase.groupby('Sector').size()
        labels = base2.index
        ax1.pie(x=base2.values, autopct="%.1f%%", explode=[0.05]*len(base2.values), labels=labels, pctdistance=0.5)
        plt.title("Proporción de # contratos");
        ax2=fig.add_subplot(gs[1,1])
        base2=fbase.groupby('Sector').sum()['Valor del Contrato']
        labels = base2.index
        ax2.pie(x=base2.values, autopct="%.1f%%", explode=[0.05]*len(base2.values), labels=labels, pctdistance=0.5)
        plt.title("Proporción de Valor de contratos");
    except:
        alignment = {'horizontalalignment': 'center', 'verticalalignment': 'center'}
        plt.figtext(0.5, 0.7,'En este Departamento no invirtieron en ese sector',family='monospace',  size='xx-large', **alignment)
        plt.axis(False)
    return 
interact(sectorvsdeptovsanio,
         i=widgets.ToggleButtons(options=df['Departamento'].unique(),
                            description="Dpto:",button_style='info'
                        ),
         j=widgets.SelectMultiple(
             options=df['Sector'].unique(),
             description='Sector:'))

## Caso(6)  Red de Investigadores y Flujos Migratorios

Tomado de [*From Data to Viz*](https://www.data-to-viz.com/story/AdjacencyMatrix.html)
La matriz de incidencia y adyacencia proporciona una relación entre varios nodos. La información que contienen puede tener diferente naturaleza, por lo que este documento considerará varios ejemplos:

Las relaciones se pueden dirigir y ponderar. Como la cantidad de personas que migran de un país a otro. Los datos utilizados provienen de esta [publicación científica](https://onlinelibrary.wiley.com/doi/abs/10.1111/imre.12327) de [Gui J. Abel.](https://guyabel.com/)


In [0]:
DF=pd.read_csv("https://raw.githubusercontent.com/holtzy/data_to_viz/master/Example_dataset/13_AdjacencyDirectedWeighted.csv",sep=" ")
DF

Las relaciones pueden no estar dirigidas ni ponderadas. Considere a todos los coautores de un investigador y estudio que está conectado a través de una publicación común. La matriz de adyacencia con alrededor de 100 investigadores, llena de 1 si han publicado un artículo juntos, 0 en caso contrario:

In [0]:
DF2=pd.read_csv("https://raw.githubusercontent.com/holtzy/Google-Scholar-Network/master/DATA/ranwez_adj.csv",sep=" ",index_col="from")
DF2=DF2.fillna(0) 
DF2

### Diagrama Sankey

Los diagramas de Sankey visualizan las contribuciones a un flujo definiendo la fuente para representar el nodo de origen, el objetivo para el nodo de destino, el valor para establecer el volumen de flujo y la etiqueta que muestra el nombre del nodo.

In [0]:
DF['Africa']

In [0]:
sour_target=[]
parallel=[]
for i,name_i in enumerate(DF.index):
    for j,name_j in enumerate(DF.columns):
        sour_target.append([i,j,DF[name_j][name_i]])
        parallel.extend([[name_i,name_j]]*int(DF[name_j][name_i]*1000) )
DFSTV=pd.DataFrame(data=sour_target,columns=['Source','Target','Value'])
DFP=pd.DataFrame(data=parallel,columns=['Source','Target'])

In [0]:
DFP

In [0]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = DF.columns
    ),
    link = dict(
      source = DFSTV['Source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = DFSTV['Target'],
      value = DFSTV['Value']
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [0]:
fig = px.parallel_categories(DFP)
fig.show()


### Mapa de calor
También resulta útil para determinar conexiones, elemental pero elegante:

In [0]:
sns.heatmap(DF,annot=True)

In [0]:
for i in set(DF2.index).difference(set(DF2.columns)):
    DF2[i]=0
for i in set(DF2.columns).difference(set(DF2.index)):
    DF2.loc[i]=0
    
DF2

## Gráfos

Los grafos son naturales para expresar conexiones, algo complicado de pintar pero aquí dejamos un ejemplo:

In [0]:
import networkx as nx
G = nx.from_pandas_adjacency(DF2)

In [0]:
G.edges;

In [0]:
from networkx import path_graph, random_layout
pos = random_layout(G, seed=42) 

In [0]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Viridis',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(str(adjacencies[0])+' Total connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [0]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Grafo hecho en Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Grafo de conecciones entre investigadores",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

## Intermezzo Gráficos interesantes de plotly

Veremos a continuación unalista de gráficos útiles de plotly.


### Dispersion 3d

En ocasiones resulta muy útil utilizar una dimensión adicional en nuestro gráficos. Veamos unos ejemplos:


In [0]:
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()

In [0]:
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
                    color='petal_length', symbol='species')
fig.show()

In [0]:
t = np.linspace(0, 10, 50)
x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')])
fig.show()

### Los gráficos paralelos

El diagrama de categorías paralelas (también conocido como conjuntos paralelos o diagrama aluvial) es una visualización de conjuntos de datos categóricos multidimensionales. Cada variable en el conjunto de datos está representada por una columna de rectángulos, donde cada rectángulo corresponde a un valor discreto tomado por esa variable. Las alturas relativas de los rectángulos reflejan la frecuencia relativa de aparición del valor correspondiente.

Las combinaciones de rectángulos de categoría en todas las dimensiones están conectadas por cintas, donde la altura de la cinta corresponde a la frecuencia relativa de aparición de la combinación de categorías en el conjunto de datos.



In [0]:
df = px.data.tips()
fig = px.parallel_categories(df)

fig.show()

Con estilo:

In [0]:

fig = px.parallel_categories(df, dimensions=['sex', 'smoker', 'day'],
                color="size", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

Escogiendo colores:

In [0]:
titanic_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv")

# Create dimensions
class_dim = go.parcats.Dimension(
    values=titanic_df.Pclass,
    categoryorder='category ascending', label="Class"
)

gender_dim = go.parcats.Dimension(values=titanic_df.Sex, label="Gender")

survival_dim = go.parcats.Dimension(
    values=titanic_df.Survived, label="Outcome", categoryarray=[0, 1],
    ticktext=['perished', 'survived']
)

# Create parcats trace
color = titanic_df.Survived;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, survival_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

Un regalito de la documentación de plotly, para estudiarlo un poco más:

In [0]:
cars_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/imports-85.csv')
cars_df

In [0]:
# Build parcats dimensions
categorical_dimensions = ['body-style', 'drive-wheels', 'fuel-type']

dimensions = [dict(values=cars_df[label], label=label) for label in categorical_dimensions]

# Build colorscale
color = np.zeros(len(cars_df), dtype='uint8')
colorscale = [[0, 'gray'], [0.33, 'gray'],
              [0.33, 'firebrick'], [0.66, 'firebrick'],
              [0.66, 'blue'], [1.0, 'blue']]
cmin = -0.5
cmax = 2.5

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[go.Scatter(x=cars_df.horsepower, y=cars_df['highway-mpg'],
                marker={'color': color, 'cmin': cmin, 'cmax': cmax,
                        'colorscale': colorscale, 'showscale': True,
                        'colorbar': {'tickvals': [0, 1, 2], 'ticktext': ['None', 'Red', 'Blue']}},
                     mode='markers'),

      go.Parcats(domain={'y': [0, 0.4]}, dimensions=dimensions,
                   line={'colorscale': colorscale, 'cmin': cmin,
                   'cmax': cmax, 'color': color, 'shape': 'hspline'})]
)

fig.update_layout(height=800, xaxis={'title': 'Horsepower'},
                  yaxis={'title': 'MPG', 'domain': [0.6, 1]},
                  dragmode='lasso', hovermode='closest')

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Blue'],
    index=1, description='Brush Color:', disabled=False)

# Update color callback
def update_color(trace, points, state):
    # Compute new color array
    new_color = np.array(fig.data[0].marker.color)
    new_color[points.point_inds] = color_toggle.index

    with fig.batch_update():
        # Update scatter color
        fig.data[0].marker.color = new_color

        # Update parcats colors
        fig.data[1].line.color = new_color

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

# Display figure
widgets.VBox([color_toggle, fig])

### Diagramas ternarios

Una forma de combinar tres variables en un diagrama 2D

In [0]:
df = px.data.election()
df

In [0]:
fig = px.scatter_ternary(df, a="Joly", b="Coderre", c="Bergeron")
fig.show()

In [0]:
fig = px.scatter_ternary(df, a="Joly", b="Coderre", c="Bergeron", hover_name="district",
    color="winner", size="total", size_max=15,
    color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"} )
fig.show()